In [14]:
from geopy.geocoders import Nominatim
import pandas as pd


def obtener_coordenadas(direccion):
    try:
        geolocator = Nominatim(user_agent="geoapi")
        location = geolocator.geocode(direccion)
        if location:
            return location.latitude, location.longitude
        else:
            return "no se encontraron coordenadas para la dirección"
    except Exception as e:
        return f"ocurrio un error: {e}"

def coordenadas_locales():
    df = pd.read_excel("database\\locales RM.xlsx")

    for locale in df["Dirección"]:
        df["Coordenadas"] = f"{obtener_coordenadas(locale)}" # Arreglar codigo para que pueda extraer uno y guardar el dato, el problema es que guardó solo una dirección en toda la base

    return df

coordenadas_locales().to_excel("database\\pruebas.xlsx", index=False)

In [7]:
import requests
import pandas as pd

class WeatherAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5/weather"

    def get_weather(self, city):
        params = {
            'q': city,
            'appid': self.api_key,
            'units': 'metric'
        }
        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.status_code}"

    def json_to_dataframe(self, weather_json):
        if isinstance(weather_json, dict):  # Ensure we have a valid JSON response
            # Flatten the JSON structure to extract relevant data
            data = {
                'City': weather_json.get('name', 'N/A'),
                'Temperature': weather_json.get('main', {}).get('temp', 'N/A'),
                'Feels Like': weather_json.get('main', {}).get('feels_like', 'N/A'),
                'Humidity': weather_json.get('main', {}).get('humidity', 'N/A'),
                'Weather': weather_json.get('weather', [{}])[0].get('description', 'N/A'),
                'Wind Speed': weather_json.get('wind', {}).get('speed', 'N/A'),
                'Country': weather_json.get('sys', {}).get('country', 'N/A'),
            }
            # Convert the dictionary to a DataFrame
            return pd.DataFrame([data])  # Wrap data in a list to create a single-row DataFrame
        else:
            return pd.DataFrame()  # Return an empty DataFrame if the input is invalid

# Example usage:
api_key = "f89c0f541491d835bfbdd818446e621f"  # Replace with your actual OpenWeatherMap API key
weather_api = WeatherAPI(api_key)
weather_data = weather_api.get_weather("Santiago")

# Convert JSON to DataFrame
weather_df = weather_api.json_to_dataframe(weather_data)
weather_df
# To get historical weather data, we need to use a different endpoint or API that provides historical data.
# OpenWeatherMap provides a One Call API that includes historical weather data, but it requires a different endpoint and parameters.

class WeatherAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.openweathermap.org/data/2.5/weather"
        self.one_call_url = "http://api.openweathermap.org/data/2.5/onecall/timemachine"

    def get_weather(self, city):
        params = {
            'q': city,
            'appid': self.api_key,
            'units': 'metric'
        }
        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.status_code}"

    def get_historical_weather(self, lat, lon, dt):
        params = {
            'lat': lat,
            'lon': lon,
            'dt': dt,
            'appid': self.api_key,
            'units': 'metric'
        }
        response = requests.get(self.one_call_url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return f"Error: {response.status_code}"

    def json_to_dataframe(self, weather_json):
        if isinstance(weather_json, dict):  # Ensure we have a valid JSON response
            # Flatten the JSON structure to extract relevant data
            data = {
                'City': weather_json.get('name', 'N/A'),
                'Temperature': weather_json.get('main', {}).get('temp', 'N/A'),
                'Feels Like': weather_json.get('main', {}).get('feels_like', 'N/A'),
                'Humidity': weather_json.get('main', {}).get('humidity', 'N/A'),
                'Weather': weather_json.get('weather', [{}])[0].get('description', 'N/A'),
                'Wind Speed': weather_json.get('wind', {}).get('speed', 'N/A'),
                'Country': weather_json.get('sys', {}).get('country', 'N/A'),
            }
            # Convert the dictionary to a DataFrame
            return pd.DataFrame([data])  # Wrap data in a list to create a single-row DataFrame
        else:
            return pd.DataFrame()  # Return an empty DataFrame if the input is invalid

# Example usage:
api_key = "f89c0f541491d835bfbdd818446e621f"  # Replace with your actual OpenWeatherMap API key
weather_api = WeatherAPI(api_key)
weather_data = weather_api.get_weather("Santiago")

# Convert JSON to DataFrame
weather_df = weather_api.json_to_dataframe(weather_data)

# Get historical weather data for Santiago (using coordinates from the current weather data)
lat = weather_data['coord']['lat']
lon = weather_data['coord']['lon']
# Example timestamp for historical data (Unix timestamp for a specific date)
timestamp = 1633035600  # Replace with the desired timestamp

historical_weather_data = weather_api.get_historical_weather(lat, lon, timestamp)
historical_weather_df = weather_api.json_to_dataframe(historical_weather_data)
historical_weather_df

""


In [ ]:
print(historical_weather_df)